# REGRESIÓN MÚLTIPLE

In [58]:
import math #operaciones matemáticas
import numpy as np #gestión numérica
import pandas as pd #estructuras de datos
import matplotlib.pyplot as plt #gráficos
import seaborn as sns #gráficos
from sklearn.model_selection import train_test_split as tts #modelar
from sklearn.linear_model import LinearRegression #regresión lineal
lr = LinearRegression()

## Paso 0: Obtención de datos.

### Carga de datos:

Para el ejemplo del TALLER 2, se seleccionaron todos los datos.

In [ ]:
datos = "datos_regresion_full_send.csv" #archivo con los datos
db = pd.read_csv(datos, sep=';') #we're in
db.head() #vista previa de los datos

## Paso 1: Estadística descriptiva univariada.

### Exploración de los datos:

In [ ]:
db.describe() #estadísticas de variables cuantitativas

Con este resumen estadístico podemos realizar un pequeño analisis del comportamiento de las variables:

VENTAS: La media (52.58) es menor a la mediana (54.50) ==> la dist. tiende a la izquierda.
-rango: (68 - 30) ==> 38 mill.
-std: En promedio los valores de las ventas se alejan o desvían 12.29 mill. de la media.

PRECIO: La media (10.58) es mayor a la mediana (10.00) ==> la dist. tiende a la derecha.
-rango: (15 - 8) ==> 7 mil.
-std: En promedio los valores del precio se alejan o desvían 1.9 mil de la media.

P_COMP: La media (12.08) es mayor a la mediana (12.00) ==> la dist. tiende a la derecha.
-rango: (15 -10) ==> 5 mil.
-std: En promedio los valores del precio de compra se alejan o desvían 1.33 mil de la media.

GASTOS_PB: La media (330.83) es menor a la mediana (335.50) ==> la dist. tiende a la izquierda.
-rango: (352 - 280) ==> 72 mil.
-std: En promedio los valores de gastos en publicidad se alejan o desvían 31.04 mil de la media.

INGRESOS: La media (743.83) es menor a la mediana (757.00) ==> la dist. tiende a la izquierda.
-rango: (882 - 557) ==> 325 mil.
-std: En promedio los valores de gastos en publicidad se alejan o desvían 121.64 mil de la media.


In [ ]:
#visualización de cada variable en histogramas y boxplots

plt.figure(figsize=(50, 20))

#histograma ventas
plt.subplot(2,5,1)
plt.hist(db['VENTAS'])
plt.title('Histograma Ventas')

#histograma precio
plt.subplot(2,5,2)
plt.hist(db['PRECIO'])
plt.title('Histograma Precio')

#histograma p_comp
plt.subplot(2,5,3)
plt.hist(db['P_COMP'])
plt.title('Histograma Precio de compra')

#histograma gastos_pb
plt.subplot(2,5,4)
plt.hist(db['GASTOS_PB'])
plt.title('Histograma Gastos en publicidad')

#histograma ingresos
plt.subplot(2,5,5)
plt.hist(db['INGRESOS'])
plt.title('Histograma Ingresos')

#boxplot ventas
plt.subplot(2,5,6)
sns.boxplot(x = db["VENTAS"]).set_title("Boxplot Ventas")

#boxplot precio
plt.subplot(2,5,7)
sns.boxplot(x = db["PRECIO"]).set_title("Boxplot Precio")

#boxplot p_comp
plt.subplot(2,5,8)
sns.boxplot(x = db["P_COMP"]).set_title("Boxplot Precio de compra")

#boxplot gastos_pb
plt.subplot(2,5,9)
sns.boxplot(x = db["GASTOS_PB"]).set_title("Boxplot Gastos en publicidad")

#boxplot ingresos
plt.subplot(2,5,10)
sns.boxplot(x = db["INGRESOS"]).set_title("Boxplot Ingresos")

Con estos gráficos podemos realizar el siguiente análisis:

Todas las distribuciones son unimodales, excepto las de gastos en publicidad e ingresos, esas son bimodales.

Las distribuciones de precio y precio de compra tienen valores atípicos en 15 mil. Atípicamente habrán precios de 15 mil.

### Cálculo de coeficiente de variación (CV):

In [ ]:
ventas = db['VENTAS']
precio = db['PRECIO']
p_comp = db['P_COMP']
gastos_pb = db['GASTOS_PB']
ingresos = db['INGRESOS']

#cálculo coeficientes de variación
cv = lambda x: np.std(x, ddof = 1 ) / np.mean(x) * 100
calculo_cv = input('Ingrese la variable: 1 = {}; 2 = {}; 3 = {}; 4 = {}; 5 = {}'.format(db.columns[0],
                                                                                        db.columns[1],
                                                                                        db.columns[2],
                                                                                        db.columns[3],
                                                                                        db.columns[4]))

#interpretación coeficientes de variación
if calculo_cv == '1':
    cv = cv(ventas)
    name = db.columns[0]
    print('Coeficiente de variación: {} (x) = {:.2f}%'.format(name, cv))
elif calculo_cv == '2':
    cv = cv(precio)
    name = db.columns[1]
    print('Coeficiente de variación: {} (x) = {:.2f}%'.format(name, cv))
elif calculo_cv == '3':
    cv = cv(p_comp)
    name = db.columns[2]
    print('Coeficiente de variación: {} (x) = {:.2f}%'.format(name, cv))
elif calculo_cv == '4':
    cv = cv(gastos_pb)
    name = db.columns[3]
    print('Coeficiente de variación: {} (x) = {:.2f}%'.format(name, cv))
elif calculo_cv == '5':
    cv = cv(ingresos)
    name = db.columns[4]
    print('Coeficiente de variación: {} (x) = {:.2f}%'.format(name, cv))

if cv <= 30:
   print('Los valores de la variable {} son homogéneos.\nMUESTRA REPRESENTATIVA.'.format(name))
elif cv > 30 and cv < 50:
   print('Los valores de la variable {} son un poco heterogéneos.\nMUESTRA POCO REPRESENTATIVA.'.format(name))
elif cv > 50:
   print('Los valores de la variable {} son muy heterógeneos.\nMUESTRA NO REPRESENTATIVA.'.format(name))

## Paso 2: Búsqueda de correlaciones.

En este caso intentaremos predecir los INGRESOS (Y) de la compañía, dados las VENTAS (X1) y los GASTOS_PB (X2).

In [ ]:
#búsqueda de correlaciones dada la variable INGRESO
print(db.corr()['INGRESOS']) #correlaciones entre Y y X's

In [ ]:
#unidades de variable INGRESO dadas en miles de pesos
unidad = 1000 #miles
nombre_unidad = 'pesos' #de pesos

Con un mapa de calor se pueden observar las correlaciones entre todas las variables. 

In [ ]:
#sns.pairplot(data = db, height = 1.5)
sns.heatmap(db.corr(), annot = True)
plt.show()

## Paso 3: Modelar los datos.

In [ ]:
#db_reg = db.copy() #realizo una copia de los datos para no alterar la db original
db_reg = db[['VENTAS', 'PRECIO', 'P_COMP', 'GASTOS_PB', 'INGRESOS']]

def ln(x): #creación de valores ln para cada columna
    return math.log(x)

db_reg['VENTAS_ln'] = db_reg['VENTAS'].apply(ln)
db_reg['PRECIO_ln'] = db_reg['PRECIO'].apply(ln)
db_reg['P_COMP_ln'] = db_reg['P_COMP'].apply(ln)
db_reg['GASTOS_PB_ln'] = db_reg['GASTOS_PB'].apply(ln)
db_reg['INGRESOS_ln'] = db_reg['INGRESOS'].apply(ln)

In [ ]:
#nombrando cada columna de la base de datos
X = db_reg.loc[:, ['VENTAS',
                   'PRECIO',
                   'P_COMP',
                   'GASTOS_PB']].values #variables independientes (Xn)

Y = db_reg.loc[:, ['INGRESOS']].values #variable dependiente (Y)

X_log = db_reg.loc[:, ['VENTAS_ln',
                       'PRECIO_ln',
                       'P_COMP_ln',
                       'GASTOS_PB_ln']].values #variables independientes alt_log (Xn)

Y_log = db_reg.loc[:, ['INGRESOS_ln']].values #variable dependiente alt_log (Y)

In [ ]:
print(lr.fit(X, Y))
print(lr.score(X, Y))
print(lr.coef_)
print(lr.intercept_)

¡AQUI VAMOS!

In [ ]:
#mejor R2
R2s = []

#R2_lineal
lr.fit(X, Y)
R2_lineal = lr.score(X, Y)
R2s.append((R2_lineal, 'LINEAL'))

#R2_log
lr.fit(X_log, Y)
R2_lineal = lr.score(X_log, Y)
R2s.append((R2_lineal, 'LOGARÍTMICA'))

#R2_exp
lr.fit(X, Y_log)
R2_lineal = lr.score(X, Y_log)
R2s.append((R2_lineal, 'EXPONENCIAL'))

#R2_pot
lr.fit(X_log, Y_log)
R2_lineal = lr.score(X_log, Y_log)
R2s.append((R2_lineal, 'POTENCIAL'))

for r2 in R2s:
    print(r2)

max_R2 = max(R2s, key=lambda x: x[0]) #recomendación para modelo
print('\nRECOMENDACIÓN: El mejor R2 ({:.2f}) lo arroja una regresión: {}.'.format(max_R2[0],
                                                                                max_R2[1]))

In [ ]:
#eleccion de variables y nombres ¡¡¡IMPORTANTE!!!
ingresos = db_reg['INGRESOS']
ventas = db_reg['VENTAS']
gastos = db_reg['GASTOS_PB']

nombre_x = db.columns[4] #en este ejemplo, Y siempre serán los ingresos

variable_dependiente = input('Escoja la variable dependiente: 1 = Consumo; 2 = Precio: ')
#plotear histogramas de variables juntos
plt.figure(figsize= (12, 5))
if variable_dependiente == '1':
    nombre_y = 'Consumo'
    #ingreso
    plt.subplot(1,2,1)
    plt.hist(x = ingreso, bins=15)
    plt.title('Ingreso')

    #consumo
    plt.subplot(1,2,2)
    plt.hist(x = consumo, bins=15)
    plt.title('Consumo')
elif variable_dependiente == '2':
    nombre_y = 'Precio'
    #ingreso
    plt.subplot(1,2,1)
    plt.hist(x = ingreso, bins=15)
    plt.title('Ingreso')

    #precio
    plt.subplot(1,2,2)
    plt.hist(x = precio, bins=15)
    plt.title('Precio')

print('X = {}.'.format(nombre_x))
print('Y = {}.'.format(nombre_y))

In [ ]:
#mejor R2
R2s = []

if variable_dependiente == '1':
    #R2_ingreso_consumo_lineal
    ingreso_consumo_lineal = lr.fit(X, y_consumo)
    R2_consumo_lineal = lr.score(X, y_consumo)
    R2s.append((R2_consumo_lineal, 'LINEAL'))

    #R2_ingreso_consumo_logaritmico
    ingreso_consumo_lineal = lr.fit(X_log, y_consumo)
    R2_consumo_log = lr.score(X_log, y_consumo)
    R2s.append((R2_consumo_log, 'LOGARÍTMICO'))

    #ingreso_consumo_exponencial
    ingreso_consumo_exp = lr.fit(X, y_consumo_log)
    R2_consumo_exp = lr.score(X, y_consumo_log)
    R2s.append((R2_consumo_exp, 'EXPONENCIAL'))

    #ingreso_consumo_potencial
    ingreso_consumo_pot = lr.fit(X_log, y_consumo_log)
    R2_consumo_pot = lr.score(X_log, y_consumo_log)
    R2s.append((R2_consumo_pot, 'POTENCIAL'))

    ###---DIAGRAMA DE DISPERSIÓN---###
    #plt.title('Diagrama de dispersión: {} / {}.'.format(nombre_x.lower(),
    #                                                    nombre_y.lower()))
    #plt.xlabel('(x) {}'.format(nombre_x))
    #plt.ylabel('(y) {}'.format(nombre_y))
    #plt.scatter(X, y_consumo) #gráfico de dispersión
    #plt.show() #for clean output
    
    #join ingreso/consumo
    plt.figure(figsize = (12,10))
    plt.subplot(2,2,2)
    plt.scatter(x = ingreso, y = consumo)
    plt.title('Dispersión: Ingreso / Consumo')

    #distibución de ingreso
    plt.subplot(2,2,4)
    plt.hist(x = ingreso, bins = 15)
    plt.title('Distribución de Ingreso')

    #distibución de consumo
    plt.subplot(2,2,1)
    plt.hist(x = consumo, orientation ='horizontal', bins = 15)
    plt.title('Distribución de Consumo')
    plt.show()

elif variable_dependiente == '2':
    #ingreso_consumo_lineal
    ingreso_precio_lineal = lr.fit(X, y_precio)
    R2_precio_lineal = lr.score(X, y_precio)
    R2s.append((R2_precio_lineal, 'LINEAL'))

    #ingreso_consumo_logaritmico
    ingreso_precio_lineal = lr.fit(X_log, y_precio)
    R2_precio_log = lr.score(X_log, y_precio)
    R2s.append((R2_precio_log, 'LOGARÍTMICO'))

    #ingreso_consumo_exponencial
    ingreso_precio_exp = lr.fit(X, y_precio_log)
    R2_precio_exp = lr.score(X, y_precio_log)
    R2s.append((R2_precio_exp, 'EXPONENCIAL'))

    #ingreso_consumo_potencial
    ingreso_precio_pot = lr.fit(X_log, y_precio_log)
    R2_precio_pot = lr.score(X_log, y_precio_log)
    R2s.append((R2_precio_pot, 'POTENCIAL'))
    ###---DIAGRAMA DE DISPERSIÓN---###
    #plt.title('Diagrama de dispersión: {} / {}.'.format(nombre_x.lower(),
    #                                                    nombre_y.lower()))
    #plt.xlabel('(x) {}'.format(nombre_x))
    #plt.ylabel('(y) {}'.format(nombre_y))
    #plt.scatter(X, y_precio) #gráfico de dispersión
    #plt.show() #for clean output

    #join ingreso/consumo
    plt.figure(figsize = (12,10))
    plt.subplot(2,2,2)
    plt.scatter(x = ingreso, y = precio)
    plt.title('Dispersión: Ingreso / Precio')
    
    #distibución de ingreso
    plt.subplot(2,2,4)
    plt.hist(x = ingreso, bins = 15)
    plt.title('Distribución de Ingreso')
    
    #distibución de consumo
    plt.subplot(2,2,1)
    plt.hist(x = precio, orientation ='horizontal', bins = 15)
    plt.title('Distribución de Precio')
    plt.show()

max_R2 = max(R2s, key=lambda x: x[0]) #recomendación para modelo
print('RECOMENDACIÓN: El mejor R2 ({:.2f}) para el modelo {} / {} lo arroja una regresión: {}.'.format(max_R2[0],
                                                                                                   nombre_x.lower(),
                                                                                                   nombre_y.lower(),
                                                                                                   max_R2[1]))

## Paso 3: Elección, estimación e interpretación del modelo.

In [ ]:
modelo = input ('Escoja un modelo: 1 = Lineal; 2 = Logarítmico; 3 = Exponencial; 4 = Potencial: ')

if variable_dependiente == '1':

    if modelo == '1':
        #elección_ingreso/consumo_lineal
        ingreso_consumo_lineal = lr.fit(X, y_consumo)
        R2_consumo_lineal = lr.score(X, y_consumo)
        print('REGRESIÓN LINEAL: {} / {}.'.format(nombre_x.lower(),
                                                  nombre_y.lower()))
        print('R^2 = {:.2f}'.format(R2_consumo_lineal))
        a_lin = lr.intercept_
        b_lin = lr.coef_
        print('\nA = {:.2f}'.format(a_lin[0]))
        #interpretación_ingreso/consumo_lineal
        print('Si {} es constante, {} es igual a {:.2f} {}.'.format(nombre_x.lower(),
                                                                    nombre_y.lower(),
                                                                    a_lin[0] * unidad,
                                                                    nombre_unidad))
        print('\nB = {:.5f}'.format(b_lin[0][0]))
        print('Por un incremento de {:.2f} {} en {}, el {} aumenta en {:.5f} {}.'.format(unidad,
                                                                                         nombre_unidad,
                                                                                         nombre_x.lower(),
                                                                                         nombre_y.lower(),
                                                                                         b_lin[0][0] * unidad,
                                                                                         nombre_unidad))
        print('\nECUACIÓN: Y = {:.2f} + {:.5f}X'.format(a_lin[0],b_lin[0][0]))
        #estimación_ingreso/consumo_lineal
        print('\nY-ESTIMADA:')
        y_pred = lr.predict(X)
        print(y_pred[0:5]) #primeros 5 valores estimados

    if modelo == '2':
        #elección_ingreso/consumo_log
        ingreso_consumo_lineal = lr.fit(X_log, y_consumo)
        R2_consumo_log = lr.score(X_log, y_consumo)
        print('REGRESIÓN LOGARÍTMICA: {} / {}.'.format(nombre_x.lower(),
                                                       nombre_y.lower()))
        print('\nR^2 = {:.2f}'.format(R2_consumo_log))
        a_log = lr.intercept_
        b_log = lr.coef_
        print('\nA = {:.2f}'.format(a_log[0]))
        print('\nB = {:.5f}'.format(b_log[0][0]))
        #interpretación_ingreso/consumo_log
        print('Por un incremento del 1% en {}, el {} aumenta en {:.5f} {}.'.format(nombre_x.lower(),
                                                                                   nombre_y.lower(),
                                                                                   b_log[0][0] * unidad,
                                                                                   nombre_unidad))
        print('\nECUACIÓN: Y = {:.2f} + {:.5f}LnX'.format(a_log[0],b_log[0][0]))
        #estimación_ingreso/consumo_log
        print('\nY-ESTIMADA:')
        y_pred = lr.predict(X)
        print(y_pred[0:5]) #primeros 5 valores estimados

    if modelo == '3':
        #elección_ingreso/consumo_exp
        ingreso_consumo_exp = lr.fit(X, y_consumo_log)
        R2_consumo_exp = lr.score(X, y_consumo_log)
        print('REGRESIÓN EXPONENCIAL: {} / {}.'.format(nombre_x.lower(),
                                                       nombre_y.lower()))
        print('\nR^2 = {:.2f}'.format(R2_consumo_exp))
        a_exp = lr.intercept_
        b_exp = lr.coef_
        print('\nA = {:.2f}'.format(a_exp[0]))
        print('\nB = {:.5f}'.format(b_exp[0][0]))
        #interpretación_ingreso/consumo_exp
        print('Por un incremento de {:.2f} {} en {}, el {} aumenta en {:.5f} {}.'.format(unidad,
                                                                                         nombre_unidad,
                                                                                         nombre_x.lower(),
                                                                                         nombre_y.lower(),
                                                                                         b_exp[0][0] * unidad,
                                                                                         nombre_unidad))
        print('\nECUACIÓN: Y = Ln[e^{:.2f} * e^{:.7f}X]'.format(a_exp[0],b_exp[0][0]))
        #estimación_ingreso/consumo_exp
        print('\nY-ESTIMADA:')
        y_pred = lr.predict(X)
        print(y_pred[0:5]) #primeros 5 valores estimados

    if modelo == '4':
        #elección_ingreso/consumo_pot
        ingreso_consumo_pot = lr.fit(X_log, y_consumo_log)
        R2_consumo_pot = lr.score(X_log, y_consumo_log)
        print('REGRESIÓN POTENCIAL: {} / {}.'.format(nombre_x.lower(),
                                                  nombre_y.lower()))
        print('\nR^2 = {:.2f}'.format(R2_consumo_pot))
        a_pot = lr.intercept_
        b_pot = lr.coef_
        print('\nA = {:.2f}'.format(a_pot[0]))
        print('\nB = {:.5f}'.format(b_pot[0][0]))
        #interpretación_ingreso/consumo_pot
        print('Por un incremento del 1% en {}, el {} aumenta en {:.2f}%.'.format(nombre_x.lower(),
                                                                                     nombre_y.lower(),
                                                                                     b_pot[0][0] * 100))
        print('\nECUACIÓN: Y = Ln[e^{:.2f} * X^({:.7f}LnX)]'.format(a_pot[0],b_pot[0][0]))
        #estimación_ingreso/consumo_pot
        print('\nY-ESTIMADA:')
        y_pred = lr.predict(X)
        print(y_pred[0:5]) #primeros 5 valores estimados
        
elif variable_dependiente == '2':

    if modelo == '1':
        #elección_ingreso/precio_lineal
        ingreso_precio_lineal = lr.fit(X, y_precio)
        R2_precio_lineal = lr.score(X, y_precio)
        print('REGRESIÓN LINEAL: {} / {}.'.format(nombre_x.lower(),
                                                  nombre_y.lower()))
        print('R^2 = {:.2f}'.format(R2_precio_lineal))
        a_lin = lr.intercept_
        b_lin = lr.coef_
        print('\nA = {:.2f}'.format(a_lin[0]))
        #interpretación_ingreso/precio_lineal
        print('Si {} es constante, {} es igual a {:.2f} {}.'.format(nombre_x.lower(),
                                                                    nombre_y.lower(),
                                                                    a_lin[0] * unidad,
                                                                    nombre_unidad))
        print('\nB = {:.5f}'.format(b_lin[0][0]))
        print('Por un incremento de {:.2f} {} en {}, el {} disminuye en {:.5f} {}.'.format(unidad,
                                                                                         nombre_unidad,
                                                                                         nombre_x.lower(),
                                                                                         nombre_y.lower(),
                                                                                         b_lin[0][0] * unidad,
                                                                                         nombre_unidad))
        print('\nECUACIÓN: Y = {:.2f} + {:.5f}X'.format(a_lin[0],b_lin[0][0]))
        #estimación_ingreso/precio_lineal
        print('\nY-ESTIMADA:')
        y_pred = lr.predict(X)
        print(y_pred[0:5]) #primeros 5 valores estimados
    
    if modelo == '2':
        #elección_ingreso/precio_log
        ingreso_precio_lineal = lr.fit(X_log, y_precio)
        R2_precio_log = lr.score(X_log, y_precio)
        print('REGRESIÓN LOGARÍTMICA: {} / {}.'.format(nombre_x.lower(),
                                                       nombre_y.lower()))
        print('\nR^2 = {:.2f}'.format(R2_precio_log))
        a_log = lr.intercept_
        b_log = lr.coef_
        print('\nA = {:.2f}'.format(a_log[0]))
        print('\nB = {:.5f}'.format(b_log[0][0]))
        #interpretación_ingreso/precio_log
        print('Por un incremento del 1% en {}, el {} disminuye en {:.5f} {}.'.format(nombre_x.lower(),
                                                                                   nombre_y.lower(),
                                                                                   b_log[0][0] * unidad,
                                                                                   nombre_unidad))
        print('\nECUACIÓN: Y = {:.2f} + {:.5f}LnX'.format(a_log[0],b_log[0][0]))
        #estimación_ingreso/precio_log
        print('\nY-ESTIMADA:')
        y_pred = lr.predict(X)
        print(y_pred[0:5]) #primeros 5 valores estimados

    if modelo == '3':
        #elección_ingreso/precio_exp
        ingreso_precio_exp = lr.fit(X, y_precio_log)
        R2_precio_exp = lr.score(X, y_precio_log)
        print('REGRESIÓN EXPONENCIAL: {} / {}.'.format(nombre_x.lower(),
                                                       nombre_y.lower()))
        print('\nR^2 = {:.2f}'.format(R2_precio_exp))
        a_exp = lr.intercept_
        b_exp = lr.coef_
        print('\nA = {:.2f}'.format(a_exp[0]))
        print('\nB = {:.5f}'.format(b_exp[0][0]))
        #interpretación_ingreso/precio_exp
        print('Por un incremento de {:.2f} {} en {}, el {} disminuye en {:.5f} {}.'.format(unidad,
                                                                                         nombre_unidad,
                                                                                         nombre_x.lower(),
                                                                                         nombre_y.lower(),
                                                                                         b_exp[0][0] * unidad,
                                                                                         nombre_unidad))
        print('\nECUACIÓN: Y = Ln[e^{:.2f} * e^{:.7f}X]'.format(a_exp[0],b_exp[0][0]))
        #estimación_ingreso/precio_exp
        print('\nY-ESTIMADA:')
        y_pred = lr.predict(X)
        print(y_pred[0:5]) #primeros 5 valores estimados

    if modelo == '4':
        #elección_ingreso/precio_pot
        ingreso_precio_pot = lr.fit(X_log, y_precio_log)
        R2_precio_pot = lr.score(X_log, y_precio_log)
        print('REGRESIÓN POTENCIAL: {} / {}.'.format(nombre_x.lower(),
                                                  nombre_y.lower()))
        print('\nR^2 = {:.2f}'.format(R2_precio_pot))
        a_pot = lr.intercept_
        b_pot = lr.coef_
        print('\nA = {:.2f}'.format(a_pot[0]))
        print('\nB = {:.5f}'.format(b_pot[0][0]))
        #interpretación_ingreso/precio_pot
        print('Por un incremento del 1% en {}, el {} disminuye en {:.2f}%.'.format(nombre_x.lower(),
                                                                                     nombre_y.lower(),
                                                                                     b_pot[0][0] * 100))
        print('\nECUACIÓN: Y = Ln[e^{:.2f} * X^({:.7f}LnX)]'.format(a_pot[0],b_pot[0][0]))
        #estimación_ingreso/precio_pot
        print('\nY-ESTIMADA:')
        y_pred = lr.predict(X)
        print(y_pred[0:5]) #primeros 5 valores estimados

## Paso 4: Pronóstico.

In [ ]:
def estimar_valor_lineal():
    print('PRONÓSTICO PARA REGRESIÓN LINEAL:\n')

    x = []
    y = []
    
    x_estimado = None
    while x_estimado != 0:
        x_estimado = float(input('Ingrese un valor que desee estimar; (0) para salir: '))
        if x_estimado != 0:
            y_estimado = a_lin[0] + ((b_lin[0][0] * x_estimado))
            x.append(x_estimado)
            y.append(y_estimado)
            print('Se estima que por cada ${:.2f} {} en el {}, haya un {} de ${:.2f} {}.'.format(x_estimado,
                                                                                                 nombre_unidad,
                                                                                                 nombre_x.lower(),
                                                                                                 nombre_y.lower(),
                                                                                                 y_estimado * 1000,
                                                                                                 nombre_unidad))

def estimar_valor_log():
    print('PRONÓSTICO PARA REGRESIÓN LOGARÍTMICA:\n')

    x = []
    y = []
    
    x_estimado = None
    while x_estimado != 0:
        x_estimado = float(input('Ingrese un valor que desee estimar; (0) para salir: '))
        if x_estimado != 0:
            y_estimado = (b_log[0][0] * math.log(x_estimado)) + a_log[0]
            x.append(x_estimado)
            y.append(y_estimado)
            print('Se estima que por cada ${:.2f} {} en el {}, haya un {} de ${:.2f} {}.'.format(x_estimado,
                                                                                                 nombre_unidad,
                                                                                                 nombre_x.lower(),
                                                                                                 nombre_y.lower(),
                                                                                                 y_estimado * 1000,
                                                                                                 nombre_unidad))

def estimar_valor_exp():
    print('PRONÓSTICO PARA REGRESIÓN EXPONENCIAL:\n')

    x = []
    y = []
    
    x_estimado = None
    while x_estimado != 0:
        x_estimado = float(input('Ingrese un valor que desee estimar; (0) para salir: '))
        if x_estimado != 0:
            y_estimado = a_exp[0] + (x_estimado * b_exp[0][0])
            x.append(x_estimado)
            y.append(y_estimado)
            print('Se estima que por cada ${:.2f} {} en el {}, haya un {} de ${:.2f} {}.'.format(x_estimado,
                                                                                                 nombre_unidad,
                                                                                                 nombre_x.lower(),
                                                                                                 nombre_y.lower(),
                                                                                                 y_estimado * 1000,
                                                                                                 nombre_unidad))

def estimar_valor_pot():
    print('PRONÓSTICO PARA REGRESIÓN POTENCIAL:\n')

    x = []
    y = []
    
    x_estimado = None
    while x_estimado != 0:
        x_estimado = float(input('Ingrese un valor que desee estimar; (0) para salir: '))
        if x_estimado != 0:
            y_estimado = (math.e ** a_pot[0]) * (x_estimado ** b_pot[0][0])
            x.append(x_estimado)
            y.append(y_estimado)
            print('Se estima que por cada ${:.2f} {} en el {}, haya un {} de ${:.2f} {}.'.format(x_estimado,
                                                                                                 nombre_unidad,
                                                                                                 nombre_x.lower(),
                                                                                                 nombre_y.lower(),
                                                                                                 y_estimado * 1000,
                                                                                                 nombre_unidad))

if modelo == '1':
    estimar_valor_lineal()
elif modelo == '2':
    estimar_valor_log()
elif modelo == '3':
    estimar_valor_exp()
elif modelo == '4':
    estimar_valor_pot()

In [ ]:
###---TEST---###

#fit_lin = np.polyfit(x, y, 1) #parámetros para lineal
#print('LINEAL: pendiente (m) = {:.3f}; intersección (a) = {:.3f}.\n'.format(fit_lin[0], fit_lin[1]))
#
#fit_log = np.polyfit(x_log, y, 1) #parámetros para logarítmica
#print('LOGARÍTMICA: pendiente (m) = {:.3f}; intersección (a) = {:.3f}.\n'.format(fit_log[0], fit_log[1]))
#
#fit_exp = np.polyfit(x, y_log, 1) #parámetros para exponencial
#print('EXPONENCIAL: pendiente (m) = {:.3f}; intersección (a) = {:.3f}.\n'.format(fit_exp[0], fit_exp[1]))
#
#fit_pot = np.polyfit(x_log, y_log, 1) #parámetros para potencial
#print('POTENCIAL: pendiente (m) = {:.3f}; intersección (a) = {:.3f}.'.format(fit_pot[0], fit_pot[1]))